This is a prototype of the desired workflow of solving a QBAF that contains support links and nodes of topics and product.

In [1]:
import pandas as pd
from reasoner.models import CountinuousDFQuADModel
from reasoner.solver import Solver
from reasoner.utilities import Collector, Adaptor

In [2]:
# load data
fpath = "./data/chunks.csv"
chunks = pd.read_csv(fpath)
chunks

,argument_id,chunk,topic,rank,polarity_score
0,0,I always get a half size up in my tennis shoes .,4,0.500000,-0.166667
1,0,For some reason these feel to big in the heel ...,15,0.500000,-0.050000
2,1,walked 3 hours with no problem,8,0.248569,0.000000
3,1,Put them on and !,3,0.254298,0.000000
4,1,Love them !,2,0.246981,0.625000
...,...,...,...,...,...
1193,369,I can wear the shoe all day long and,3,0.125414,-0.050000
1194,369,they are easy to clean compared to other shoes...,0,0.126618,0.225000
1195,369,They are light colored so any dirt will be see...,18,0.126863,0.342857
1196,369,Would definitely buy another pair in a differe...,16,0.124921,0.000000


In [15]:
# remove all chunks of an review
def remove_review_by_id(review_id, chunks):
    result = chunks[chunks["argument_id"] != review_id].reset_index(drop=True)
    return result

remove_review_by_id(0, chunks)

,argument_id,chunk,topic,rank,polarity_score
0,1,walked 3 hours with no problem,8,0.248569,0.000000
1,1,Put them on and !,3,0.254298,0.000000
2,1,Love them !,2,0.246981,0.625000
3,1,So light feeling,9,0.250153,0.400000
4,2,excelente,-1,1.000000,0.000000
...,...,...,...,...,...
1191,369,I can wear the shoe all day long and,3,0.125414,-0.050000
1192,369,they are easy to clean compared to other shoes...,0,0.126618,0.225000
1193,369,They are light colored so any dirt will be see...,18,0.126863,0.342857
1194,369,Would definitely buy another pair in a differe...,16,0.124921,0.000000


In [4]:
# create node and link dataframes
def get_nodes(chunks):
    topics = chunks["topic"].unique().tolist()
    topics.remove(-1)
    topics.sort()
    topics = [f"Topic {t}" for t in topics]
    topics.append("Product")
    
    chunk_text = chunks["chunk"]
    chunk_text = pd.concat([chunk_text, pd.Series(topics)], ignore_index=True)
    
    result = pd.DataFrame({"chunk": chunk_text}) 
    result["weight"] = 0.5
    return result

def get_supports(chunks):
    topic_index_start = max(chunks.index) + 1
    topics = chunks["topic"].unique().tolist()
    topics.remove(-1)
    topics.sort()
    product_index = topic_index_start + len(topics)
   
    # add supports between chunks and topics
    supports = chunks[chunks["polarity_score"] >= 0]
    supports = supports[supports["topic"] != -1]
    supports["chunk_id"] = supports.index
    supports = supports[["chunk_id", "topic"]]
    supports = supports.reset_index(drop=True)
    supports["topic"] += topic_index_start
    supports = supports.rename(columns={
        "chunk_id": "source", 
        "topic": "target"
    })
    
    # add support between topics and product
    new_rows = pd.DataFrame({"source": list(range(topic_index_start, topic_index_start + len(topics)))})
    new_rows["target"] = product_index
    supports = pd.concat([supports, new_rows], ignore_index=True) 
    supports = supports.astype(int)
    return supports

def get_attacks(chunks):
    topic_index_start = max(chunks.index) + 1
   
    # add supports between chunks and topics
    attacks = chunks[chunks["polarity_score"] < 0]
    attacks = attacks[attacks["topic"] != -1]
    attacks["chunk_id"] = attacks.index
    attacks = attacks[["chunk_id", "topic"]]
    attacks = attacks.reset_index(drop=True)
    attacks["topic"] += topic_index_start
    attacks = attacks.rename(columns={
        "chunk_id": "source", 
        "topic": "target"
    })
    attacks = attacks.astype(int)
    return attacks

In [10]:
# build adaptor of input data
nodes = get_nodes(chunks)
supports = get_supports(chunks)
attacks = get_attacks(chunks)

adaptor = Adaptor(arguments=nodes, weight_col="weight", attacks=attacks, supports=supports)

In [11]:
# build solver
model = CountinuousDFQuADModel(data_adaptor=adaptor)
solver = Solver(model=model, step_size=0.5, max_steps=10e4, epsilon=10e-4)
final_step, collector = solver.solve(solver="RK4", collect_data=True)

In [12]:
base_prod_strength = model.strength_vector[-1]
model.strength_vector[-26:-1], model.strength_vector[-1]

(array([0.50097466, 0.56237793, 0.62475586, 0.50012183, 0.50000001,
        0.74951172, 0.74951172, 0.53118896, 0.50024366, 0.62475586,
        0.99902344, 0.56237793, 0.74951172, 0.51559445, 0.74951172,
        0.50000001, 0.53118896, 0.51559448, 0.74951172, 0.53117374,
        0.62475586, 0.50779723, 0.7495116 , 0.62475491, 0.56231701]),
 0.9990234374890825)

In [9]:
result = []
for i in [0, 154, 155]:
    part_chunks = remove_review_by_id(i, chunks)
    nodes = get_nodes(part_chunks)
    supports = get_supports(part_chunks)
    attacks = get_attacks(part_chunks)
    adaptor = Adaptor(arguments=nodes, weight_col="weight", attacks=attacks, supports=supports)
    model = CountinuousDFQuADModel(data_adaptor=adaptor)
    solver = Solver(model=model, step_size=0.5, max_steps=10e4, epsilon=10e-4)
    final_step, _ = solver.solve(solver="RK4", collect_data=False)
    print(f"Remove review #{i}, covergence reached at step {final_step}: {model.strength_vector[-26:-1]}, {model.strength_vector[-1]}")

Remove review #0, covergence reached at step 8: [0.50097466 0.56237793 0.62475586 0.50012183 0.50000003 0.74951172
 0.74951172 0.53118896 0.50024366 0.62475586 0.99902344 0.56237793
 0.74951172 0.51559445 0.74951172 0.50000001 0.53118896 0.51559448
 0.74951172 0.53117374 0.62475586 0.50779723 0.7495116  0.62475491
 0.56231701], 0.9990234374890825
Remove review #154, covergence reached at step 8: [0.50097466 0.56237793 0.62475586 0.50012183 0.50000001 0.74951172
 0.74951172 0.53118896 0.50024366 0.62475586 0.99902344 0.56237793
 0.74951172 0.51559445 0.74951172 0.50000001 0.53118896 0.51559448
 0.74951172 0.53117374 0.62475586 0.50779723 0.7495116  0.62475396
 0.56231701], 0.9990234374890825
Remove review #155, covergence reached at step 8: [0.50097466 0.56237793 0.62475586 0.50012183 0.50000001 0.74951172
 0.74951172 0.53118896 0.50024366 0.62475586 0.99902344 0.56237793
 0.74951172 0.51559445 0.74951172 0.50000001 0.53118896 0.51559448
 0.74951172 0.53117374 0.62475586 0.50779721 0.74